In [1]:
from collections import Counter, defaultdict

from estnltk import Span, Layer, Text
from estnltk.converters import text_to_json, json_to_text
from estnltk.taggers import VabamorfTagger

import json
import math
import nltk
import os
import re
import string 

In [2]:
#os.chdir(os.getcwd() + "\..")

In [3]:
# Loeb emotikonid sisse
emotikonid = []

for failinimi in ["wikipedia_emoticons_list.txt", "Unicode_emoticons_list.txt", "looks.wtf.txt", "unicode_emojis.txt"]:
    with open("../Loendid/emotikonid/"+failinimi, "r", encoding="UTF-8") as fr:
        for line in fr.readlines():
            # Väiketähestab
            emotikonid.append(line.strip().lower())
# Eemaldab korduvad emotikonid
emotikonid = list(set(emotikonid))

# Mõned emotikonid võivad olla ka kokkukleepumise tõttu olla väärpositiivsed (":pole")
emotikonid_probleemsed = []
with open("../Loendid/emotikonid/wikipedia_emoticons_sp.txt", "r", encoding="UTF-8") as fr:
    for line in fr.readlines():
        # Väiketähestab
        emotikonid_probleemsed.append(line.strip().lower())
# Eemaldab korduvad emotikonid 
emotikonid_probleemsed = list(set(emotikonid_probleemsed))

In [4]:
oletamisega_morph_tagger = VabamorfTagger(guess=True, propername=True, disambiguate=True)

In [5]:
# Kaustade nimed
source = "../etnc19_web_2019_100000/"
guess_destination = "../etnc19_web_2019_morf_oletamisega/"

# Loob kaustad
os.makedirs(os.path.dirname(guess_destination), exist_ok=True)

In [6]:
# Loob sõnastiku keskmiste arvutamiseks
keskmiste_arvutamiseks = defaultdict(int)

# Avab järjest kõik failid algkaustas
for filename in [f for f in os.listdir(source)]:
    
    with open(os.path.join(source, filename), "r", encoding="UTF-8") as f:

        faili_keskmised = defaultdict(int)
        
        # Loeb failist vaid sisu, ignoreerib alguses olevat metainfot
        pure = "".join(f.readlines()[1:])
        
        # Regexiga leiab kõik potentsiaalsed emojid tekstist (kahe kooloni vaheline whitespaceita tekst)
        emojid = re.findall(":\S+?:", pure)
        # Eemaldab leitud emojide hulgast väärvasted
        wrong = [":http:", ":https:"]
        for value in wrong:
            while value in emojid:
                emojid.remove(value)
        # Eemaldab emojid tekstist, et nende sisu ei peetaks sõnadeks 
        for emoji in emojid:
            pure = pure.replace(emoji, "")

        # Otsib tekstist emotikone ja paneb need nimekirja
        emotikonid_leitud = []
        for emotikon in emotikonid:
            emotikonid_leitud.extend(re.findall(re.escape(emotikon), pure, re.IGNORECASE))
            # Eemaldab tekstist leitud emotikonid, et need sõnestamisel lahku löömisel need keskmiseid ei mõjutaks
            pure = re.sub(re.escape(emotikon), '', pure, re.IGNORECASE)
        # Vaatab tekstist eraldi emotikone, mis võivad olla ka kokkukleepumise tulemusel väärpositiivsed vasted
        # Lisaks eemaldad leitud emotikonid
        sobivad = []
        for emotikon in emotikonid_probleemsed:
            # Kui "silmad" on viimane emotikoni osa, kontrollib, et emotikoni ees ei oleks tegu tähemärgiga ehk et poleks seoses sõnaga
            if emotikon[-1] == ":":
                sobivad.extend(re.findall(re.escape(emotikon), "\n".join(re.findall("\W"+re.escape(emotikon), pure, re.IGNORECASE)), re.IGNORECASE))
                pure = re.sub("(\W)"+re.escape(emotikon), '\1', pure, re.IGNORECASE)
            # Vastasel juhul kontrollib seda emotikoni lõpust
            else:
                sobivad.extend(re.findall(re.escape(emotikon), "\n".join(re.findall(re.escape(emotikon)+"\W", pure, re.IGNORECASE)), re.IGNORECASE))
                pure = re.sub(re.escape(emotikon)+"(\W)", '\1', pure, re.IGNORECASE)
        
        # Jätab meelde emotikonide arvud ja loendid vigade kontrollimiseks
        faili_keskmised['emotikonide_arv']=len(emojid) + len(emotikonid_leitud) + len(sobivad)
        faili_keskmised['emojid_loend']=emojid
        faili_keskmised['emotikonid_loend']=emotikonid_leitud
        faili_keskmised['emotikonid_erilised_loend']=sobivad
        
        # Teeb morfoloogilise analüüsi tundmatude analüüsi oletamisega
        oletamisega = Text(pure)
        
        oletamisega.tag_layer(['words', 'sentences', 'compound_tokens'])
        
        oletamisega_morph_tagger.tag( oletamisega )
        
        # Loob salvestamiseks failinimed
        ga_name = ".".join(filename.split(".")[:-1]) + "_morf_oletamisega.json"
    
        
        # Loeb kokku lemmade arvud ja käänduvate lemmade arvud
        kõikide_lemmade_arv = 0
        ainult_käänduvate_lemmade_arv = 0
        
        for lemma, postag in zip(oletamisega.lemma, oletamisega.partofspeech):
            kõikide_lemmade_arv += 1
            keskmiste_arvutamiseks['kõikide_lemmade_arv'] += 1
            # Kui tegu on käänduva lemmaga
            # Vaatab ainult esimest sõnaliiki (et välistada käändelsi verbivorme omadussõnade hulgast)
            if postag[0] in ["A", "C", "G", "H", "K", "N", "O", "P", "S", "U", "Y"]:
                ainult_käänduvate_lemmade_arv += 1
                keskmiste_arvutamiseks['ainult_käänduvate_lemmade_arv'] += 1

        lemmas_subwords = []
        for tokens in oletamisega.root_tokens:
            lemmad = None
            # Võtab lemmade loendist esimese tõlgenduse:
            lemmad = tokens[0]
            # Vaatab iga lemmat tekstis
            for lemma in lemmad:
                # Kui kõik tähemärgid ei ole punktuatsioonimärgid
                if not all(char in string.punctuation for char in lemma):
                    lemmas_subwords.append(lemma)
                        
        # Jätab meelde osasõnade pikkused ja arvu
        keskmiste_arvutamiseks['lemmade_osasõnade_pikkused'] += sum(map(len, lemmas_subwords))
        keskmiste_arvutamiseks['lemmade_osasõnade_arv'] += len(lemmas_subwords)
        
        # Võtab sõnade algvormid, ignoreerib kirjavahemärke
        lemmad = [lemma[0] for lemma in oletamisega.lemma  if not all(char in string.punctuation for char in lemma)]
        
        # Arvutab TTR-i, keskmise lemma osasõna pikkuse ja käänduvate lemmade osaarvu
        faili_keskmised['TTR'] = len(Counter(lemmad))/len(lemmad)
        faili_keskmised['keskmine_lemma_pikkus'] = sum(map(len, lemmas_subwords))/len(lemmas_subwords)
        faili_keskmised['käänduvate_lemmade_osaarv'] = ainult_käänduvate_lemmade_arv/kõikide_lemmade_arv
        
        # Liidab TTR-id üle failide kokku, et leida tekstide keskmine TTR
        keskmiste_arvutamiseks['TTR_keskmine'] += faili_keskmised['TTR']
        
        for key, value in faili_keskmised.items():
            oletamisega.meta[key] = value
        
        json_text = text_to_json(oletamisega, file = os.path.join(guess_destination, ga_name))

In [7]:
# Loob keskmiste sõnastiku
keskmised = defaultdict(float)

# Arvutab keskmised
keskmised['TTR'] = keskmiste_arvutamiseks['TTR_keskmine']/100000
keskmised['keskmine_lemma_pikkus'] = keskmiste_arvutamiseks['lemmade_osasõnade_pikkused']/keskmiste_arvutamiseks['lemmade_osasõnade_arv']
keskmised['käänduvate_lemmade_osaarv'] = keskmiste_arvutamiseks['ainult_käänduvate_lemmade_arv']/keskmiste_arvutamiseks['kõikide_lemmade_arv']

# Salvestab keskmised faili
with open("keskmised.json", "w", encoding = "utf-8") as fw: 
    json.dump(keskmised, fw, sort_keys=True, indent=4)